# Lesson 2: Building and Training a CNN for Image Classification

## Introduction

In this notebook, we will build and train a Convolutional Neural Network (CNN) to classify images from the CIFAR-10 and CIFAR-100 datasets. We'll follow the typical data science pipeline:

1. Load and explore the data
2. Analyze the dataset
3. Implement data augmentation
4. Build a CNN model
5. Train and evaluate the model
6. Experiment with different architectures
7. Scale to a more complex dataset (CIFAR-100)

Let's get started!

## 1. Setup and Data Loading

First, let's import the necessary libraries:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import random

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check if GPU is available
# Check for CUDA (NVIDIA GPU) or MPS (Apple Silicon) availability
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Now, let's load the CIFAR-10 dataset:

In [ ]:
# Define basic transformations for loading the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

# Create data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                        shuffle=False, num_workers=2)

# Define the classes in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

Let's visualize some of the images from the dataset:

In [ ]:
# Function to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(10, 4))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

# Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images[:8]))
print('Labels:', ' '.join(f'{classes[labels[j]]:5s}' for j in range(8)))

## 2. Data Analysis

Now, let's analyze the dataset to better understand what we're working with.

In [ ]:
# TODO: Implement a function to analyze the dataset
# This function should output metrics about the dataset such as:
# - Number of samples in training and test sets
# - Number of samples per class
# - Size of one data sample (image dimensions)

def analyze_dataset(trainset, testset, classes):
    """
    Analyze the dataset and print useful information.
    
    Args:
        trainset: The training dataset
        testset: The test dataset
        classes: List of class names
    """
    # Your code here
    # Hint: You can use trainset.data to access the raw data
    
    # 1. Get the total number of samples
    
    # 2. Get the image dimensions
    
    # 3. Count samples per class in the training set
    
    # 4. Visualize the class distribution
    
    pass

# Call the function to analyze the dataset
analyze_dataset(trainset, testset, classes)

## 3. Data Augmentation

Data augmentation is a technique to artificially increase the size of the training dataset by applying various transformations to the original images. This helps improve the model's generalization ability.

In [ ]:
# TODO: Implement a function for data augmentation
# This function should apply different transformations to the images
# For more information on available transforms, see:
# https://pytorch.org/vision/stable/transforms.html

def get_augmentation_transforms():
    """
    Create a set of data augmentation transforms.
    
    Returns:
        A torchvision.transforms.Compose object with various augmentations
    """
    # Your code here
    # Hint: Use transforms like RandomHorizontalFlip, RandomRotation, ColorJitter, etc.
    
    augmentation_transforms = transforms.Compose([
        # Add your transformations here
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    return augmentation_transforms

# Create a new training dataset with augmentation
augmentation_transforms = get_augmentation_transforms()
augmented_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=augmentation_transforms)
augmented_trainloader = torch.utils.data.DataLoader(augmented_trainset, batch_size=64,
                                                  shuffle=True, num_workers=2)

Let's visualize some of the augmented images:

In [ ]:
# Get some random augmented training images
dataiter = iter(augmented_trainloader)
augmented_images, augmented_labels = next(dataiter)

# Show original images
imshow(torchvision.utils.make_grid(images[:4]))
print('Original Labels:', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

# Show augmented images
imshow(torchvision.utils.make_grid(augmented_images[:4]))
print('Augmented Labels:', ' '.join(f'{classes[augmented_labels[j]]:5s}' for j in range(4)))

## 4. Building a Simple CNN

Now, let's build a simple CNN model for image classification.

In [ ]:
# Here's a minimal example of a CNN with one convolutional layer and a fully connected layer
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 2, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 16, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 -> 16x16
        x = x.view(-1, 16 * 16)
        x = F.relu(self.fc1(x))
        return x

# Create an instance of the model
model = SimpleCNN(num_classes=10).to(device)
print(model)
# infer one data with the model
dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to(device)
outputs = model(images)
_, predicted = torch.max(outputs.data, 1)
print(f"Predicted: {predicted}")


## 5. Training the Model

Now, let's implement the training loop for our CNN model.

In [ ]:
# TODO: Implement the training loop
# This function should train the model on the training data and evaluate it on the test data

# Documentation links:
# - PyTorch optimizers: https://pytorch.org/docs/stable/optim.html
# - PyTorch loss functions: https://pytorch.org/docs/stable/nn.html#loss-functions

def train_model(model, trainloader, testloader, criterion, optimizer, num_epochs=5):
    """
    Train the model.
    
    Args:
        model: The neural network model
        trainloader: DataLoader for training data
        testloader: DataLoader for test data
        criterion: Loss function
        optimizer: Optimization algorithm
        num_epochs: Number of training epochs
    
    Returns:
        Lists of training losses, training accuracies, and test accuracies
    """
    # Your code here
    # Hint: A typical training loop includes:
    # 1. Iterate through batches of training data
    # 2. Zero the gradients
    # 3. Forward pass
    # 4. Compute loss
    # 5. Backward pass
    # 6. Update weights
    # 7. Evaluate on test data
    
    # Initialize lists to track metrics
    train_losses = []
    train_accuracies = []
    test_accuracies = []
    
    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Your training code here
        
        # Evaluation phase
        model.eval()
        test_correct = 0
        test_total = 0
        
        # Your evaluation code here
        
        # Print statistics
        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {train_loss:.4f}, '
              f'Train Acc: {train_acc:.2f}%, '
              f'Test Acc: {test_acc:.2f}%')
        
        # Store metrics
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)
    
    return train_losses, train_accuracies, test_accuracies

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
train_losses, train_accuracies, test_accuracies = train_model(
    model, augmented_trainloader, testloader, criterion, optimizer, num_epochs=5)

Let's visualize the training progress:

In [ ]:
# Plot training and test metrics
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train')
plt.plot(test_accuracies, label='Test')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

## 6. Visualizing Learned Filters

Let's visualize the filters (weights) learned by the convolutional layer:

In [ ]:
# Get the weights of the first convolutional layer
weights = model.conv1.weight.data.cpu()

# Plot the filters
plt.figure(figsize=(12, 6))
num_filters = weights.shape[0]  # Get actual number of filters

# Create a grid for visualization
rows = 2
cols = 3
for i in range(num_filters):
    if i >= rows * cols:
        break  # Limit the number of filters to display
    
    plt.subplot(rows, cols, i+1)
    
    # For RGB filters, we'll use a single channel representation for simplicity
    # We'll use the average of the 3 channels
    filter_data = weights[i].mean(dim=0).numpy()
    
    # Display the filter
    im = plt.imshow(filter_data, cmap='viridis')
    plt.axis('off')
    plt.title(f'Filter {i+1}')

# Add a colorbar with gradient legend showing actual weight values
cax = plt.axes([0.92, 0.2, 0.02, 0.6])  # [left, bottom, width, height]
cmap = plt.cm.viridis
norm = plt.Normalize(weights.min().item(), weights.max().item())
cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
cbar.set_label('Weight Value')

plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Adjust layout to make room for colorbar
plt.show()

## 7. Improving the Model

Now, let's try to improve our model by adding more layers and features.

In [ ]:
# TODO: Implement an improved CNN model
# This model should have more convolutional layers and possibly other improvements

class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ImprovedCNN, self).__init__()
        # Your code here
        # Hint: Consider adding more convolutional layers, batch normalization,
        # dropout, or other techniques to improve the model
        
        pass
    
    def forward(self, x):
        # Your code here
        pass

# Create an instance of the improved model
improved_model = ImprovedCNN(num_classes=10).to(device)
print(improved_model)

# Define loss function and optimizer for the improved model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(improved_model.parameters(), lr=0.001)

# Train the improved model
improved_train_losses, improved_train_accuracies, improved_test_accuracies = train_model(
    improved_model, augmented_trainloader, testloader, criterion, optimizer, num_epochs=10)

Let's compare the performance of our simple and improved models:

In [ ]:
# Plot comparison of test accuracies
plt.figure(figsize=(10, 6))
plt.plot(test_accuracies, label='Simple CNN')
plt.plot(improved_test_accuracies, label='Improved CNN')
plt.title('Test Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)
plt.show()

## 8. Scaling to CIFAR-100

Now, let's adapt our model to work with the more complex CIFAR-100 dataset, which has 100 classes instead of 10.

In [ ]:
# Load CIFAR-100 dataset
cifar100_trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                                download=True, transform=augmentation_transforms)
cifar100_testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                               download=True, transform=transform)

cifar100_trainloader = torch.utils.data.DataLoader(cifar100_trainset, batch_size=64,
                                                 shuffle=True, num_workers=2)
cifar100_testloader = torch.utils.data.DataLoader(cifar100_testset, batch_size=64,
                                                shuffle=False, num_workers=2)

# TODO: Adapt your improved model for CIFAR-100
# You'll need to modify the final layer to output 100 classes instead of 10

# Create an instance of the model for CIFAR-100
cifar100_model = ImprovedCNN(num_classes=100).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cifar100_model.parameters(), lr=0.001)

# Train the model on CIFAR-100
cifar100_train_losses, cifar100_train_accuracies, cifar100_test_accuracies = train_model(
    cifar100_model, cifar100_trainloader, cifar100_testloader, criterion, optimizer, num_epochs=10)

## 9. Final Evaluation and Comparison

Let's evaluate our models on the test sets and compare their performance:

In [ ]:
# Function to evaluate a model on a dataset
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

# Evaluate models
simple_cifar10_accuracy = evaluate_model(model, testloader, device)
improved_cifar10_accuracy = evaluate_model(improved_model, testloader, device)
cifar100_accuracy = evaluate_model(cifar100_model, cifar100_testloader, device)

print(f"Simple CNN on CIFAR-10: {simple_cifar10_accuracy:.2f}% accuracy")
print(f"Improved CNN on CIFAR-10: {improved_cifar10_accuracy:.2f}% accuracy")
print(f"Improved CNN on CIFAR-100: {cifar100_accuracy:.2f}% accuracy")

# Plot final comparison
accuracies = [simple_cifar10_accuracy, improved_cifar10_accuracy, cifar100_accuracy]
models = ['Simple CNN\n(CIFAR-10)', 'Improved CNN\n(CIFAR-10)', 'Improved CNN\n(CIFAR-100)']

plt.figure(figsize=(10, 6))
plt.bar(models, accuracies, color=['blue', 'green', 'red'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)
for i, v in enumerate(accuracies):
    plt.text(i, v + 1, f"{v:.2f}%", ha='center')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## 10. Conclusion

In this notebook, we've gone through the entire data science pipeline for image classification:

1. We loaded and explored the CIFAR-10 and CIFAR-100 datasets
2. We analyzed the datasets to understand their characteristics
3. We implemented data augmentation to improve model generalization
4. We built a simple CNN model and trained it on CIFAR-10
5. We visualized the learned filters to understand what the model is learning
6. We improved our model with additional layers and techniques
7. We adapted our model to work with the more complex CIFAR-100 dataset
8. We compared the performance of our different models

Some key takeaways:
- CNNs are powerful for image classification tasks
- Data augmentation helps improve model generalization
- Deeper models can capture more complex patterns
- As the number of classes increases (CIFAR-10 to CIFAR-100), the classification task becomes more challenging

## 11. Bonus Challenges

If you've completed the notebook and want to explore further, here are some bonus challenges:

1. Try implementing a state-of-the-art CNN architecture like ResNet or EfficientNet
2. Experiment with different learning rates and optimizers
3. Implement learning rate scheduling to improve training
4. Use transfer learning with a pre-trained model
5. Implement and compare different data augmentation techniques
6. Visualize the feature maps at different layers of your CNN
7. Implement and use techniques like Grad-CAM to visualize what your model is focusing on 